In [19]:
import serial 
import struct
import time

RETURN_TEMP   = 0x01
RETURN_SAMPLE = 0x02
RETURN_HELLO  = 0x03
SET_BIAS      = 0x04
RETURN_BATT   = 0x05
TOGGLE_RED    = 0x06

serialport = '/dev/ttyUSB0' #if using the dev board
serialport = '/dev/ttyACM0' #if using the v0.1 board

serialport = 'COM4'

def getFloatValue(ser, cmd):
    assert(cmd == RETURN_TEMP or cmd == RETURN_SAMPLE or cmd == RETURN_BATT)
    ser.write(bytes([cmd, 0x00]))
    data = ser.read(4)
    return struct.unpack('>i', data)[0]/100

def testConnection(ser):
    ser.write(bytes([RETURN_HELLO, 0x00]))
    data = ser.read(8)
    return data.strip().decode('utf-8') 

def setBias(ser, bias):
    assert(bias >= 0 and bias <= 255)
    ser.write(bytes([SET_BIAS, bias]))
    data = ser.read(8)
    return data.strip().decode('utf-8') 

In [5]:
with serial.Serial(serialport, 115200, timeout=2) as ser:
  print(testConnection(ser))

  #loop to get batt voltage with 100ms delay
  for i in range(10):
    print(getFloatValue(ser, RETURN_BATT))
    time.sleep(0.1)
  

461s058:
19370069.62
19530665.99
5423984.96
19530663.41
19134572.04
9092104.16
9260386.14
11453238.34
5401613.32
2187725.34


In [21]:
with serial.Serial(serialport, 115200, timeout=2) as ser:
  #read the temperature in C, set the bias, and read the sipm, return bias to 0
  print(testConnection(ser))
  print(getFloatValue(ser, RETURN_TEMP))
  print(setBias(ser, 0x01))
  print(getFloatValue(ser, RETURN_SAMPLE))
  print(setBias(ser, 0))
  print(getFloatValue(ser, RETURN_BATT))

hi
25.7
OK
3206.43
OK
5.19


In [6]:
with serial.Serial(serialport, 115200, timeout=2) as ser:
  ser.write(bytes([TOGGLE_RED, 0x00]))